In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn import linear_model

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [17]:
dirc = os.getcwd()
dirc

'/Users/darrendong/Desktop/KG/house-prices-advanced-regression-techniques'

In [18]:
os.listdir()

['.DS_Store',
 'Untitled.ipynb',
 'test.csv',
 'submission.csv',
 'data_description.txt',
 'train.csv',
 '.ipynb_checkpoints',
 'sample_submission.csv']

In [19]:
train = pd.read_csv(dirc+'/train.csv')
test = pd.read_csv(dirc+'/test.csv')

In [20]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [21]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [27]:
train_11 = train.drop(columns=['Alley', 'FireplaceQu','PoolQC','Fence','MiscFeature'])
train_11

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [39]:
train1 = pd.get_dummies(train_11, dummy_na = True)

In [40]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 310 entries, Id to SaleCondition_nan
dtypes: float64(3), int64(35), uint8(272)
memory usage: 821.4 KB


In [47]:
train1_X1 = train1.drop(columns =['Id', 'SalePrice'])
train1_Y1 = train1['SalePrice']

train1_X = train1_X1.fillna(train1_X1.mean())
train1_Y = train1_Y1.fillna(train1_Y1.mean())

train1_X.shape,train1_Y.shape

((1460, 308), (1460,))

In [48]:
rfecv = RFECV(estimator = DecisionTreeClassifier())
rfecv.fit(train1_X, train1_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


RFECV(estimator=DecisionTreeClassifier())

In [49]:
rfecv.n_features_

8

In [50]:
result = pd.DataFrame(data = rfecv.cv_results_)
result.head()

,mean_test_score,std_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,0.003425,0.003752,0.000000,0.003425,0.010274,0.000000,0.003425
1,0.009589,0.002563,0.006849,0.006849,0.010274,0.010274,0.013699
2,0.005479,0.005564,0.003425,0.000000,0.010274,0.000000,0.013699
3,0.004795,0.001678,0.003425,0.006849,0.006849,0.003425,0.003425
4,0.008219,0.001678,0.010274,0.006849,0.006849,0.006849,0.010274


In [51]:
np.mean(result.std_test_score)

0.004129222382096131

In [21]:
criteria = rfecv.support_

In [22]:
train2 = train1_X.loc[:, criteria]
train2

,MSSubClass,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,OverallCond,YearBuilt,...,Fireplaces,GarageYrBlt,GarageFinish,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold,SaleType
0,5,36,327,3,4,5,5,6,4,104,...,0,89,1,2,220,0,49,1,2,8
1,0,51,498,3,2,24,2,5,7,77,...,1,62,1,2,149,187,0,4,1,8
2,5,39,702,0,4,5,5,6,4,102,...,1,87,1,2,260,0,30,8,2,8
3,6,31,489,0,0,6,5,6,4,19,...,1,84,2,3,284,0,24,1,0,8
4,5,55,925,0,2,15,5,7,4,101,...,1,86,1,3,380,118,70,11,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5,33,267,3,4,8,5,5,4,100,...,1,85,1,2,149,0,28,7,1,8
1456,0,56,866,3,4,14,2,5,5,79,...,2,64,2,2,182,214,0,1,4,8
1457,6,37,415,3,4,6,5,6,8,44,...,2,29,1,1,24,0,48,4,4,8
1458,0,39,505,3,4,12,2,4,5,51,...,0,36,2,1,19,224,0,3,4,8


In [23]:
lis = []
for col in train2.columns:
    lis.append(col)

In [24]:
test1 = test[lis]
test1

,MSSubClass,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,OverallCond,YearBuilt,...,Fireplaces,GarageYrBlt,GarageFinish,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold,SaleType
0,20,80.0,11622,Reg,Inside,NAmes,1Story,5,6,1961,...,0,1961.0,Unf,1.0,730.0,140,0,6,2010,WD
1,20,81.0,14267,IR1,Corner,NAmes,1Story,6,6,1958,...,0,1958.0,Unf,1.0,312.0,393,36,6,2010,WD
2,60,74.0,13830,IR1,Inside,Gilbert,2Story,5,5,1997,...,1,1997.0,Fin,2.0,482.0,212,34,3,2010,WD
3,60,78.0,9978,IR1,Inside,Gilbert,2Story,6,6,1998,...,1,1998.0,Fin,2.0,470.0,360,36,6,2010,WD
4,120,43.0,5005,IR1,Inside,StoneBr,1Story,8,5,1992,...,0,1992.0,RFn,2.0,506.0,0,82,1,2010,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,Reg,Inside,MeadowV,2Story,4,7,1970,...,0,NaN,NaN,0.0,0.0,0,0,6,2006,WD
1455,160,21.0,1894,Reg,Inside,MeadowV,2Story,4,5,1970,...,0,1970.0,Unf,1.0,286.0,0,24,4,2006,WD
1456,20,160.0,20000,Reg,Inside,Mitchel,1Story,5,7,1960,...,1,1960.0,Unf,2.0,576.0,474,0,9,2006,WD
1457,85,62.0,10441,Reg,Inside,Mitchel,SFoyer,5,5,1992,...,0,NaN,NaN,0.0,0.0,80,32,7,2006,WD


In [25]:
reg = linear_model.Lasso(alpha=0.1)
reg.fit(train2, train1_Y)

Lasso(alpha=0.1)

In [26]:
train2.shape, train1_Y.shape

((1460, 41), (1460,))

In [27]:
test1.shape

(1459, 41)

In [28]:
test2 = test1.apply(LabelEncoder().fit_transform)

In [29]:
result = reg.predict(test2)
result

array([133.85284522, 266.51636887, 361.83789084, ..., 317.3672748 ,
       134.37133228, 479.0150401 ])

In [30]:
result.shape

(1459,)

In [31]:
id_ = []
for i in range (1461,1461+1459):
    id_.append(i)
    

In [34]:
d = {'Id': id_, 'SalePrice':result}

In [36]:
export = pd.DataFrame(data = d)

In [39]:
export.to_csv('submission.csv',index=False)